In [1]:
import requests
import json
import pandas as pd
import math
import os
import time
import re
import undetected_chromedriver as uc

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from glob import glob
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from lxml import html

In [7]:
driver = uc.Chrome()
wait = WebDriverWait(driver, 10)

In [23]:
def google_search(query):
	# Initialize the driver and DataFrame
	links_df = pd.DataFrame(columns=['title', 'url'])
	
	try:
		# Navigate to Google
		driver.get('https://www.google.com')
		
		# Find and fill the search box
		search_box = wait.until(EC.presence_of_element_located((By.NAME, "q")))
		search_box.send_keys(query)
		search_box.submit()
		
		# Wait for results and get all result elements
		#results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.g")))
		results = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//h3")))
		
		# Extract titles and URLs
		for result in results:
			try:
				#title_element = result.find_element(By.XPATH, "./h3")
				#title_element = result.find_element(By.CSS_SELECTOR, "h3")
				link_element = result.find_element(By.XPATH, ".//parent::a")
				
				title = result.text
				print(title)
				url = link_element.get_attribute('href')
				
				# Append to DataFrame
				links_df.loc[len(links_df)] = [title, url]
				
			except Exception as e:
				print(f"Error extracting link info: {e}")
				continue
		
		# Visit each URL in the DataFrame
		for url in links_df['url']:
			try:
				driver.execute_script("window.open(arguments[0]);", url)
				time.sleep(2)  # Small delay between visits
			except Exception as e:
				print(f"Error visiting URL {url}: {e}")
				continue
				
		return links_df
				
	except Exception as e:
		print(f"An error occurred: {e}")
		return links_df
	
	finally:
		time.sleep(5)  # Wait before closing

In [24]:
def scrape_links(ekatalog_links):
    for url in ekatalog_links['url']:
        try:
            # Open URL
            driver.get(url)
            
            # Wait for specs link and click
            specs_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[href="#spesifikasi"]')))
            specs_link.click()
            
            # Scroll to bottom gradually
            total_height = int(driver.execute_script("return document.body.scrollHeight"))
            for height in range(0, total_height, 100):
                driver.execute_script(f"window.scrollTo(0, {height});")
                time.sleep(0.3)
            
            # Reset to top
            driver.execute_script("window.scrollTo(0, 0);")
            
            # Wait for specs link and click
            ongkir_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[href="#ongkir"]')))
            ongkir_link.click()
            
            # Second scroll - to bottom gradually
            total_height = int(driver.execute_script("return document.body.scrollHeight"))
            for height in range(0, total_height, 100):
                driver.execute_script(f"window.scrollTo(0, {height});")
                time.sleep(0.3)

            time.sleep(2)  # Wait for page to settle
            
        except Exception as e:
            print(f"Error processing {url}: {e}")
            continue

In [25]:
# Example usage
search_query = "ups 100 kva inurl:e-katalog.lkpp.go.id"
ekatalog_links = google_search(search_query)

scrape_links(ekatalog_links)

UPS Online 100KVA | E-Katalog 5.0 - LKPP
YSP LT 100 UPS (100KVA/90KW) - e-Katalog - LKPP
UPS KSTAR UID 1000 L (100 KVA) | E-Katalog 5.0 - LKPP
UPS 100KVA | E-Katalog 5.0 - LKPP
yoshiga ups gpi 33-100 100 kva - e-Katalog
UPS 100 KVA / 80 KW | E-Katalog 5.0 - LKPP
UPS Online Three Phase ONH-100KVA/33 (Long Run Backup)
UPS 100KVA DS3100T | E-Katalog 5.0 - LKPP
UPS 100 KVA | E-Katalog 5.0 - LKPP
ONLINE UPS SIMA PLUS-100KVA | E-Katalog 5.0 - LKPP
Error processing https://e-katalog.lkpp.go.id/katalog/produk/detail/77096845: Message: 
Stacktrace:
	GetHandleVerifier [0x00DE02D3+25139]
	(No symbol) [0x00D6B284]
	(No symbol) [0x00C404A3]
	(No symbol) [0x00C88338]
	(No symbol) [0x00C8868B]
	(No symbol) [0x00CD0F62]
	(No symbol) [0x00CACF84]
	(No symbol) [0x00CCE6E1]
	(No symbol) [0x00CACD36]
	(No symbol) [0x00C7BD29]
	(No symbol) [0x00C7D064]
	GetHandleVerifier [0x010EB193+3215603]
	GetHandleVerifier [0x0110230A+3310186]
	GetHandleVerifier [0x010FC522+3286146]
	GetHandleVerifier [0x00E79CD0+65438

In [26]:
driver.quit()